In [ ]:
#Import data and play

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from scipy import stats



In [ ]:
#ran morph analysis in two batches, combine outputs
#load
data = np.load('results.npy', allow_pickle=True).item() #upload data from morphology analysis


In [ ]:
# convert to pandas dataframe
rows = []
for image_file, image_data in data.items():
    base = {
        'Image_Name': image_data['Image_Name'],
        'Image_Tag': image_data['Image_Tag'],
    }
    info = image_data['info']
    # Merge the top-level and info dictionaries
    combined = {**base, **info}
    rows.append(combined)

# Convert to DataFrame
df = pd.DataFrame(rows)


In [ ]:
#adding sphere stuff i forgot and removing clathrin 

df['Sphere_Volume'] = (4/3) * np.pi * df['sphere_radius']**3
df['Sphere_Surface_Area'] = 4 * np.pi * df['sphere_radius']**2

df = df[~df['Image_Tag'].str.contains('Clath', na=False)]

In [ ]:
#add manual label of vesicle vs bud

excel_df = pd.read_excel('vesiclesvsbuds.xlsx')
df = df.merge(excel_df, on='Image_Name', how='left') 

In [ ]:
# Select the appropriate volume based on the Ellipsoid/Sphere choice
df['Selected_Volume'] = df.apply(
    lambda row: row['Ellipsoid_Volume_nm'] if row['Ellipsoid/Sphere'] == 'Ellipsoid' else row['Sphere_Volume'],
    axis=1
)

# Select the appropriate rmse based on the Ellipsoid/Sphere choice
df['Selected_RMSE'] = df.apply(
    lambda row: row['ellipsoid_rmse'] if row['Ellipsoid/Sphere'] == 'Ellipsoid' else row['sphere_rmse'],
    axis=1
)

# Select the appropriate sphericity based on the Ellipsoid/Sphere choice
df['Selected_sphericity'] = df.apply(
    lambda row: row['sphericity_index'] if row['Ellipsoid/Sphere'] == 'Ellipsoid' else 1,
    axis=1
)

In [ ]:
#figuring out volume error for ellipsoid and sphere

In [ ]:
def compute_R_and_volume_error(row):
    # unpack ellipsoid semi-axes (a,b,c)
    a, b, c = row['ellipsoid_axes']
    
    # average radius
    R = np.mean([a, b, c])
    
    # calculate volume error estimate
    volume_error = row['Ellipsoid_Volume'] * 3 * (row['ellipsoid_rmse'] / R)
    
    return pd.Series({'R': R, 'Ellipsoid_Volume_Error': volume_error})


In [ ]:
df[['R', 'Ellipsoid_Volume_Error']] = df.apply(compute_R_and_volume_error, axis=1)



In [ ]:
def compute_sphere_R_and_volume_error(row):
    R = row['sphere_radius']  # radius of the sphere
    volume_error = row['Sphere_Volume'] * 3 * (row['sphere_rmse'] / R)
    return pd.Series({'Sphere_Volume_Error': volume_error})

In [ ]:
df[['Sphere_Volume_Error']] = df.apply(compute_sphere_R_and_volume_error, axis=1)


In [ ]:
print(df)

In [ ]:
# Filter the row with the given image_name
image_name = 'Position_14_memb_99_1-COP1.mrc'
row = df[df['Image_Name'] == image_name]
print(row)
# if not row.empty:
#     rmse_value = row['ellipsoid_rmse'].values[0]  # get the rmse value
#     print(f"eRMSE for {image_name}: {rmse_value}")
#     rmse_value = row['sphere_rmse'].values[0]  # get the rmse value
#     print(f"sRMSE for {image_name}: {rmse_value}")
# else:
#     print(f"Image name '{image_name}' not found in the DataFrame.")

In [ ]:
#Plotting

In [ ]:
palette = {'COP1': '#E4A8E2', 'COP2': '#a3d433'}

palette_2 = {'Ellipsoid': 'black', 'Sphere': 'white'}
palette_3 = {'Bud': 'black', 'Vesicle': 'white'}

palette2_errors = {'Ellipsoid': 'grey', 'Sphere': 'white'}
palette3_errors = {'Bud': 'grey', 'Vesicle': 'white'}

palette_iv = {'COP1': '#E4A8E2', 'COP2': '#a3d433','in_vitro_COP2': '#cccccc'  # grey for manual points
}



#hex codes from chimerax

In [ ]:
#All data

In [ ]:
# Scatter plot of RMSE

plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df,
    x='sphere_rmse',
    y='ellipsoid_rmse',
    hue='Image_Tag',
    palette=palette,
    s=70,
    alpha=0.8
)

# Reference y = x line
min_rmse = min(df['sphere_rmse'].min(), df['ellipsoid_rmse'].min())
max_rmse = max(df['sphere_rmse'].max(), df['ellipsoid_rmse'].max())

plt.plot([min_rmse, max_rmse], [min_rmse, max_rmse],
         linestyle='--', color='gray', label='y = x')

plt.xlabel('Sphere RMSE')
plt.ylabel('Ellipsoid RMSE')
plt.title('RMSE Comparison: Ellipsoid vs Sphere Fit')
plt.legend(title='Image Tag')
plt.grid(True)
plt.tight_layout()
plt.show()

print('Below line = Ellipsoid is the best fit')

In [ ]:
#Stacked bar chart showing number of Coated vesicles by identity and fit 

bud_categories = df['Bud/Vesicle'].unique()
image_tags = df['Image_Tag'].unique()

ct = pd.crosstab([df['Bud/Vesicle'], df['Image_Tag']], df['Ellipsoid/Sphere']).fillna(0)
index = pd.MultiIndex.from_product([bud_categories, image_tags], names=['Bud/Vesicle', 'Image_Tag'])
ct = ct.reindex(index, fill_value=0)

bar_width = 0.35
gap = 0.01
x = np.arange(len(bud_categories))

colors_dict = {
    'COP1': {'Ellipsoid': '#E4A8E2', 'Sphere': 'hotpink'},  
    'COP2': {'Ellipsoid': '#a3d433', 'Sphere': 'green'}  
}

fig, ax = plt.subplots(figsize=(10,6))

for i, tag in enumerate(image_tags):
    bottoms = np.zeros(len(bud_categories))
    counts_ellipsoid = []
    counts_sphere = []
    for bud in bud_categories:
        counts_ellipsoid.append(ct.loc[(bud, tag), 'Ellipsoid'] if 'Ellipsoid' in ct.columns else 0)
        counts_sphere.append(ct.loc[(bud, tag), 'Sphere'] if 'Sphere' in ct.columns else 0)

    counts_ellipsoid = np.array(counts_ellipsoid)
    counts_sphere = np.array(counts_sphere)

    bar_positions = x + i * (bar_width + gap)

    # Plot bars
    bars_ellipsoid = ax.bar(bar_positions, counts_ellipsoid, bar_width, label=f'{tag} Ellipsoid',
                           color=colors_dict[tag]['Ellipsoid'], bottom=bottoms)
    bottoms += counts_ellipsoid
    bars_sphere = ax.bar(bar_positions, counts_sphere, bar_width, label=f'{tag} Sphere',
                        color=colors_dict[tag]['Sphere'], bottom=bottoms)

    # Total counts for top labels
    totals = counts_ellipsoid + counts_sphere

    # Add individual counts inside bars (if bar is tall enough to display text)
    for bar_e, val_e, bar_s, val_s in zip(bars_ellipsoid, counts_ellipsoid, bars_sphere, counts_sphere):
        height_e = bar_e.get_height()
        height_s = bar_s.get_height()

        if height_e > 5:  # threshold to avoid clutter
            ax.text(
                bar_e.get_x() + bar_e.get_width() / 2,
                bar_e.get_y() + height_e / 2,
                f'{int(val_e)}',
                ha='center', va='center',
                fontsize=9, color='white', weight='bold'
            )
        if height_s > 5:
            ax.text(
                bar_s.get_x() + bar_s.get_width() / 2,
                bar_s.get_y() + height_s / 2,
                f'{int(val_s)}',
                ha='center', va='center',
                fontsize=9, color='white', weight='bold'
            )

    # Add total count above the stacked bar
    for pos, total in zip(bar_positions, totals):
        ax.text(
            pos,
            total + 2,  # small offset above the top
            f'{int(total)}',
            ha='center', va='bottom',
            fontsize=10, weight='bold',
            color='black'
        )

ax.set_xticks(x + (bar_width + gap) / 2)
ax.set_xticklabels(bud_categories)
ax.set_xlabel('Membrane Shape')
ax.set_ylabel('Count')
ax.set_title('Membrane identity, fitted shape and coat')

# Remove duplicate legend entries
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

plt.tight_layout()
plt.show()


In [ ]:
#pie charts of coated buds and vesicles 

# Filter the dataframe for COP1 and COP2 only

df_filtered = df[df['Image_Tag'].isin(['COP1', 'COP2'])]

# Group by Image_Tag and Bud/Vesicle column, count entries
counts = df_filtered.groupby(['Image_Tag', 'Bud/Vesicle']).size().unstack(fill_value=0)

print(counts) 

# Color map 
colors_map = {
    'COP1': {'Vesicle': '#E4A8E2', 'Bud': 'hotpink'},
    'COP2': {'Vesicle': '#a3d433', 'Bud': 'green'}
}

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

for i, tag in enumerate(['COP1', 'COP2']):
    sizes = counts.loc[tag]
    colors = [colors_map[tag][label] for label in sizes.index]

    wedges, texts, autotexts = axes[i].pie(
        sizes,
        labels=sizes.index,
        autopct='%1.1f%%',
        startangle=90,
        colors=colors,
        wedgeprops={'edgecolor': 'black'},
        pctdistance=1.2,      # Move percentage labels outside the pie
        labeldistance=1.05    # Optional: move category labels slightly outward too
    )

    # Make percentage labels a bit bolder and clearer
    for autotext in autotexts:
        autotext.set_fontweight('bold')
        autotext.set_color('black')

    axes[i].set_title(f'{tag}', fontweight='bold')

plt.suptitle('Bud and Vesicle Counts by Image_Tag', fontweight='bold')
plt.tight_layout()
plt.savefig("piechart.pdf", format='pdf', bbox_inches='tight')


In [ ]:
#plotting vesicle volume 

df_vesicles = df[df['Bud/Vesicle'] == 'Vesicle']

plt.figure(figsize=(10, 6))

sns.violinplot(
    data=df_vesicles,
    x='Image_Tag',
    y='Selected_Volume',
    hue='Image_Tag',
    inner='quartile',
    cut=0,
    palette=palette
)

sns.swarmplot(
    data=df_vesicles,
    x='Image_Tag',
    y='Selected_Volume',
    hue='Ellipsoid/Sphere',
    dodge=False,           # Points will overlap at the same x position
    size=4,
    # alpha=0.6,
    edgecolor='black',
    linewidth=0.5,
    palette=palette_2
)

# Remove duplicate legends
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), title='Fit Type')

plt.title('Volume Distribution of Vesicles by Image_Tag and Fit Type')
plt.xlabel('')
plt.ylabel('Volume (nm³)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig("VesicleVol.pdf", format='pdf', bbox_inches='tight')

medians = df_vesicles.groupby('Image_Tag')['Selected_Volume'].median()

print(medians)

m = df_vesicles.groupby('Image_Tag')['Selected_Volume'].max()
print(m)

m = df_vesicles.groupby('Image_Tag')['Selected_Volume'].min()
print(m)

q1 = df_vesicles.groupby('Image_Tag')['Selected_Volume'].quantile(0.25)
q3 = df_vesicles.groupby('Image_Tag')['Selected_Volume'].quantile(0.75)
print(q1)
print(q3)

filtered_df = df_vesicles[df_vesicles['Image_Tag'] == 'COP2']

# Find the row(s) with max selected_volume in that filtered DataFrame
max_row = filtered_df[filtered_df['Selected_Volume'] == filtered_df['Selected_Volume'].min()]

with pd.option_context('display.max_columns', None, 'display.width', None):

    print(max_row)


In [ ]:
#Volume of vesicles. use mann-whitney u to compare means and kolmogorov-smirnov to compare distribution

# Filter the data for COP1 and COP2 vesicles
group1 = df_vesicles[df_vesicles['Image_Tag'] == 'COP1']['Selected_Volume']
group2 = df_vesicles[df_vesicles['Image_Tag'] == 'COP2']['Selected_Volume']

# Mann-Whitney U test
u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')
print(f"Mann-Whitney U test: U={u_stat}, p={p_value_mw}")

# Kolmogorov-Smirnov test
ks_stat, p_value_ks = stats.ks_2samp(group1, group2)
print(f"Kolmogorov-Smirnov test: KS={ks_stat}, p={p_value_ks}")


In [ ]:
#Volume for buds

df_buds = df[df['Bud/Vesicle'] == 'Bud']

plt.figure(figsize=(10, 6))

sns.violinplot(
    data=df_buds,
    x='Image_Tag',
    y='Selected_Volume',
    hue='Image_Tag',
    inner='quartile',
    cut=0,
    palette=palette
)

sns.swarmplot(
    data=df_buds,
    x='Image_Tag',
    y='Selected_Volume',
    hue='Ellipsoid/Sphere',
    dodge=False,           # Points will overlap at the same x position
    size=4,
    # alpha=0.6,
    edgecolor='black',
    linewidth=0.5,
    palette=palette_2
)

# Remove duplicate legends
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), title='Fit Type')

plt.title('Volume Distribution of Buds by Image_Tag and Fit Type')
plt.xlabel('Image Tag')
plt.ylabel('Volume (nm³)')
plt.grid(True, axis='y')
plt.tight_layout()
plt.savefig("BudVol.pdf", format='pdf', bbox_inches='tight')

medians = df_buds.groupby('Image_Tag')['Selected_Volume'].median()
MAX = df_buds.groupby('Image_Tag')['Selected_Volume'].max()
min = df_buds.groupby('Image_Tag')['Selected_Volume'].min()


print(medians)
print(MAX)
print(min)

counts = df_buds.groupby(['Image_Tag', 'Ellipsoid/Sphere']).size().unstack(fill_value=0)
print(counts)


In [ ]:
#compare median volume of buds - use same methods as vesicles 

# Filter the data for COP1 and COP2 vesicles
group1 = df_buds[df_buds['Image_Tag'] == 'COP1']['Selected_Volume']
group2 = df_buds[df_buds['Image_Tag'] == 'COP2']['Selected_Volume']

# Mann-Whitney U test
u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')
print(f"Mann-Whitney U test: U={u_stat}, p={p_value_mw}")

# Kolmogorov-Smirnov test
ks_stat, p_value_ks = stats.ks_2samp(group1, group2)
print(f"Kolmogorov-Smirnov test: KS={ks_stat}, p={p_value_ks}")

In [ ]:
#plotting sphericity

In [ ]:
#sphericity of vesicles

plt.figure(figsize=(8,6))
sns.violinplot(data=df_vesicles, x='Image_Tag', y='Selected_sphericity', inner="quart", cut=0, palette=palette, hue='Image_Tag')
plt.title('Violin Plot of Vesicle Sphericity')
plt.xlabel(' ')
plt.ylabel('Sphericity index')
plt.ylim(0.0, 1.0)
plt.grid(True)

sns.swarmplot(data=df_vesicles, x='Image_Tag', y='Selected_sphericity', hue='Ellipsoid/Sphere', palette=palette_2, size=4, edgecolor='black', linewidth=0.5,)
# spread points horizontally to reduce overlap
plt.savefig("VesicleSphericity.pdf", format='pdf', bbox_inches='tight')

medians = df_vesicles.groupby('Image_Tag')['Selected_sphericity'].median()

print(medians)

In [ ]:
# Filter the data for COP1 and COP2 vesicles
group1 = df_vesicles[df_vesicles['Image_Tag'] == 'COP1']['Selected_sphericity']
group2 = df_vesicles[df_vesicles['Image_Tag'] == 'COP2']['Selected_sphericity']

# Mann-Whitney U test
u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')
print(f"Mann-Whitney U test: U={u_stat}, p={p_value_mw}")

# Kolmogorov-Smirnov test
ks_stat, p_value_ks = stats.ks_2samp(group1, group2)
print(f"Kolmogorov-Smirnov test: KS={ks_stat}, p={p_value_ks}")

In [ ]:
#sphericity of buds

plt.figure(figsize=(8,6))
sns.violinplot(data=df_buds, x='Image_Tag', y='Selected_sphericity', inner="quart", cut=0, palette=palette, hue='Image_Tag')
plt.title('Violin Plot of Bud Sphericity')
plt.xlabel(' ')
plt.ylabel('Sphericity index')
plt.ylim(0.0, 1.0)
plt.grid(True)

sns.swarmplot(data=df_buds, x='Image_Tag', y='Selected_sphericity', hue='Ellipsoid/Sphere', palette=palette_2, size=4, edgecolor='black', linewidth=0.5,)
# spread points horizontally to reduce overlap
plt.savefig("BudSphericity.pdf", format='pdf', bbox_inches='tight')

medians = df_buds.groupby('Image_Tag')['Selected_sphericity'].median()

print(medians)

In [ ]:
# Filter the data for COP1 and COP2 vesicles
group1 = df_buds[df_buds['Image_Tag'] == 'COP1']['Selected_sphericity']
group2 = df_buds[df_buds['Image_Tag'] == 'COP2']['Selected_sphericity']

# Mann-Whitney U test
u_stat, p_value_mw = stats.mannwhitneyu(group1, group2, alternative='two-sided')
print(f"Mann-Whitney U test: U={u_stat}, p={p_value_mw}")

# Kolmogorov-Smirnov test
ks_stat, p_value_ks = stats.ks_2samp(group1, group2)
print(f"Kolmogorov-Smirnov test: KS={ks_stat}, p={p_value_ks}")